In [ ]:
program shoot  % 打靶法求例题
parameter (m=2,n=100,eps=0.00001)
real(8) h, z(m,n), y(m), d(m), x1, xn, yb, alfa, beta, g1, g2
integer i,j
!open(2, file='shoot1.txt')   % 有空气阻力
open(2, file='shoot0.txt')    % 没有空气阻力
alfa = 0.0; beta = 40.0; g1 = 30.0
x1 = 0.0; xn = 5.0  % a,b
h = (xn - x1) / (n - 1)  % 步长
y(1) = alfa
y(2) = g1
call rk4(x1, y, m, h, n, z, d)
yb = z(1,n)
y(1) = alfa
y(2)=beta*g1/z(1,n)
g2=y(2)
call rk4(x1,y,m,h,n,z,d)
if (abs(z(1,n)-beta)>eps) then
   y(1)=alfa
   y(2)=g2+(beta-z(1,n))*(g2-g1)/(z(1,n)-yb)
   g1=g2
   g2=y(2)
   yb=z(1,n)
   call rk4(x1,y,m,h,n,z,d)
end if
print *, 'z(2,1)=' , z(2,1)
do i=1,n
   x=x1+(i-1)*h
   write(2,'(2f14.8)') x, z(1,i)
end do
end
subroutine f(x, y, m, d)  ! y(1): 空间高度，y(2): 速度，x: 时间
real(8) y(m), d(m), x
d(1) = y(2)
d(2) = -9.8  ! 没有空气阻力
! d(2) = -9.8 - 0.01*y(2)  ! 有空气阻力
end


### 求解量子力学一维无限深势阱**

In [ ]:
function tt = shootev()
clc; clear; format long; global k
k = 1; tol = 1e-8; m = 3;
kk = zeros(1, m); % Pre-allocate kk

for n = 1:m      % Finding the first m eigenvalues
    dk = k / 20;   % Initial step size
    k = k + dk;     % First trial
    [x, phi] = ode45(@evfun, [0, 1], [0, 1e-3]);
    oldphi = phi(end, 1);
    dphi = oldphi;

    while abs(dphi) > tol
        k = k + dk; % Try a new k
        [x, phi] = ode45(@evfun, [0, 1], [0, 1e-3]);
        dphi = phi(end, 1);

        if dphi * oldphi < 0 % Sign change: eigenvalue bracketed
            k = k - dk;      % Step back
            dk = dk / 2;     % Halve the step size (bisection)
        end
      %  oldphi = dphi;     % must remove!!
    end
    kk(n) = k;            % Store the eigenvalue
    hold on;            % Keep previous plots
    plot(x, phi(:, 1));   % Plot the eigenfunction
end

tt = kk; % Return the eigenvalues.

    function yy = evfun(x, phi)
        global k
        yy = [phi(2);          % y' = dy/dx
              -k^2 * phi(1)];    % y'' = -k²y


In [106]:
(* Global variable declaration *)
$k = 1.0; (* Initial guess for the eigenvalue. Use a floating-point number. *)

(* Eigenvalue solver function *)
shootEV[] := Module[{tol = 10^-8, m = 3, kk, n, dk, x, phi, oldphi, dphi, evFun, sol, currentGuess},

  (* Define the differential equation *)
  evFun[x_, phi_] := {phi[[2]], -$k^2*phi[[1]]};

  (* Pre-allocate array for eigenvalues *)
  kk = ConstantArray[0.0, m]; (* Initialize with floating-point numbers *)

  (* Loop to find the first 'm' eigenvalues *)
  Do[
    (* Initial step size *)
    dk = $k/20.0; (* Use a floating-point number *)
    (* First trial eigenvalue *)
    currentGuess = $k + dk;

    (* Solve the ODE with initial guess *)
    sol = Quiet@Check[
        NDSolve[{y''[x] == -currentGuess^2*y[x], y[0] == 0, y'[0] == 10^-3},
        y, {x, 0, 1}, Method -> "StiffnessSwitching"],
      $Failed,
      {NDSolve::ndsz, NDSolve::ndcf}];
    If[sol === $Failed,
      phi = {10^6, 10^6};,
      phi = y /. sol[[1]];
    ];

    (* Print the current guess *)
    (* Print["Initial guess for k: ", currentGuess]; *)

    (*Extract value for boundary condition*)
    If[Head[phi] === InterpolatingFunction,
      oldphi = phi[1],  (* Corrected:  phi[1] instead of phi[1][[1]] *)
      oldphi = 10^6.0;  (* Use a floating-point number *)
    ];

    (* Print["Old Phi", oldphi]; *)
    dphi = oldphi;

    (* Iterative refinement of the eigenvalue *)
    While[Abs[dphi] > tol,
      currentGuess = currentGuess + dk; (* Try a new k *)

      (*Solve ODE*)
      sol = Quiet@Check[
          NDSolve[{y''[x] == -currentGuess^2*y[x], y[0] == 0, y'[0] == 10^-3},
          y, {x, 0, 1}, Method -> "StiffnessSwitching"],
        $Failed,
        {NDSolve::ndsz, NDSolve::ndcf}];
      If[sol === $Failed,
        phi = {10^6, 10^6};,
        phi = y /. sol[[1]];
      ];

      (*Extract boundary condition*)
      If[Head[phi] === InterpolatingFunction,
        dphi = phi[1],  (* Corrected: phi[1] instead of phi[1][[1]] *)
        dphi = 10^6.0;   (* Use a floating-point number *)
      ];

      (* Print["dphi",dphi]; *)

      If[dphi*oldphi < 0,  (* Check for sign change *)
        currentGuess = currentGuess - dk; (* Step back *)
        dk = dk/2.0;      (* Halve the step size - Bisection. Use floating-point. *)
      ];
      oldphi = dphi;

      (* Print["Current k: ", currentGuess]; *) (* Diagnostic output *)
    ];

    kk[[n]] = currentGuess; (* Store the eigenvalue *)
    $k = currentGuess; (* Update $k for next eigenvalue search *)
    (* Print["Eigenvalue found: ", N[currentGuess, 16]]; *) (* Numerical precision *)
    , {n, 1, m}];


  (*Plot the results*)
  plot = Plot[
    Evaluate[
      Table[
        If[Head[
            y /. NDSolve[{y''[x] == -kk[[i]]^2*y[x], y[0] == 0, 
                y'[0] == 10^-3}, y, {x, 0, 1}, 
              Method -> "StiffnessSwitching"][[1]]] === 
          InterpolatingFunction,
         (y /. NDSolve[{y''[x] == -kk[[i]]^2*y[x], y[0] == 0, 
               y'[0] == 10^-3}, y, {x, 0, 1}, 
             Method -> "StiffnessSwitching"][[1]])[x], (*Corrected this \
line*)
         Null
         ],
        {i, 1, m}]
    ],
    {x, 0, 1},
    PlotRange -> All,
    AxesLabel -> {"x", "phi(x)"},
    PlotLabel -> "Eigenfunctions",
    GridLines -> Automatic,
    GridLinesStyle -> Directive[GrayLevel[0.8]]
    ];

  (* Show[plot]; *) (* Display the plot *)
  Return[kk]; (* Return the eigenvalues *)
];

(* Call the function and display results*)
eigenvalues = shootEV[];
Print["Eigenvalues (with numerical precision):"];
Print[N[eigenvalues, 16]];
$k = 1.0; (* Reset k*)
(* shootEV[];  -- No need to call it again immediately *)


Eigenvalues (with numerical precision):
{9.42461, 12.5663, 15.7079}


**初始步长太大，算法可能会跳过一个特征值而没有检测到符号变化的情况**
二分法问题
需要包围： 最大的限制。如果初始区间内没有符号变化（即特征值没有被包围），二分法将不会找到它。如果初始区间或步长太大，它可能会完全跳过特征值。需要仔细选择初始搜索范围和步长。
多重根： 如果初始区间内存在多个根，二分法只会找到其中一个（具体是哪一个取决于初始的包围情况）。
绘制边界条件函数以直观识别可能存在根的区域

In [116]:
(* 全局变量声明 *)
$k = 1.0; (* 特征值的初始猜测 *)

(* 特征值求解函数 *)
shootEV[] := Module[{tol = 10^-8, m = 3, kk, n, dk, x, phi, oldphi, dphi, evFun, sol, currentGuess, kHistory, eigenvalueMarkers},

  (* 定义微分方程 *)
  evFun[x_, phi_] := {phi[[2]], -$k^2*phi[[1]]};

  (* 预分配特征值数组 *)
  kk = ConstantArray[0.0, m];
  eigenvalueMarkers = {}; (* 用于存储特征值在图表上的位置 *)

  (* 循环查找前 'm' 个特征值 *)
  Do[
    (* 为每个特征值搜索初始化 kHistory *)
    kHistory = {};
    (* 初始步长 *)
    dk = $k/20.0;
    (* 第一个试验特征值 *)
    currentGuess = $k + dk;
    AppendTo[kHistory, currentGuess]; (* 记录初始猜测 *)

    (* 使用初始猜测求解 ODE *)
    sol = Quiet@Check[
        NDSolve[{y''[x] == -currentGuess^2*y[x], y[0] == 0, y'[0] == 10^-3},
        y, {x, 0, 1}, Method -> "StiffnessSwitching"],
      $Failed,
      {NDSolve::ndsz, NDSolve::ndcf}];
    If[sol === $Failed,
      phi = {10^6, 10^6};,
      phi = y /. sol[[1]];
    ];

    (* 提取边界条件值 *)
    If[Head[phi] === InterpolatingFunction,
      oldphi = phi[1],
      oldphi = 10^6.0;
    ];

    dphi = oldphi;

    (* 特征值的迭代细化 *)
    While[Abs[dphi] > tol,
      currentGuess = currentGuess + dk; (* 尝试新的 k *)
      AppendTo[kHistory, currentGuess]; (* 记录 k *)

      (* 求解 ODE *)
      sol = Quiet@Check[
          NDSolve[{y''[x] == -currentGuess^2*y[x], y[0] == 0, y'[0] == 10^-3},
          y, {x, 0, 1}, Method -> "StiffnessSwitching"],
        $Failed,
        {NDSolve::ndsz, NDSolve::ndcf}];
      If[sol === $Failed,
        phi = {10^6, 10^6};,
        phi = y /. sol[[1]];
      ];

      (* 提取边界条件 *)
      If[Head[phi] === InterpolatingFunction,
        dphi = phi[1],
        dphi = 10^6.0;
      ];

      If[dphi*oldphi < 0,  (* 检查符号变化 *)
        currentGuess = currentGuess - dk; (* 回退 *)
        AppendTo[kHistory, currentGuess]; (* 记录 k *)
        dk = dk/2.0;      (* 步长减半 *)
      ];
      oldphi = dphi;
    ];

    kk[[n]] = currentGuess; (* 存储特征值 *)
    $k = currentGuess; (* 更新 $k 用于下一次特征值搜索 *)
    AppendTo[eigenvalueMarkers, {currentGuess, 0}]; (* 存储标记位置 *)
   
    (* --- 绘制每个特征值的 kHistory --- *)
    Print[ListLinePlot[kHistory,
      AxesLabel -> {"迭代", "k"},
      PlotLabel -> StringJoin["特征值 ", ToString[n], " 的搜索历史"],
      PlotRange -> All,
      Epilog -> {Red, PointSize[Large], Point[{Length[kHistory], currentGuess}]} (* 标记最终的 k *)
    ]];
    
    , {n, 1, m}];


  (* --- 特征函数的组合图 --- *)
    plot = Plot[
        Evaluate[
          Table[
            If[Head[
                y /. NDSolve[{y''[x] == -kk[[i]]^2*y[x], y[0] == 0, 
                    y'[0] == 10^-3}, y, {x, 0, 1}, 
                  Method -> "StiffnessSwitching"][[1]]] === 
              InterpolatingFunction,
             (y /. NDSolve[{y''[x] == -kk[[i]]^2*y[x], y[0] == 0, 
                   y'[0] == 10^-3}, y, {x, 0, 1}, 
                 Method -> "StiffnessSwitching"][[1]])[x],
             Null
             ],
            {i, 1, m}]
        ],
        {x, 0, 1},
        PlotRange -> All,
        AxesLabel -> {"x", "phi(x)"},
        PlotLabel -> "特征函数",
        GridLines -> Automatic,
        GridLinesStyle -> Directive[GrayLevel[0.8]]
        ];
    Print[plot];

  Return[kk]; (* 返回特征值 *)
];

(* 调用函数并显示结果 *)
eigenvalues = shootEV[];
Print["特征值 (带有数值精度):"];
Print[N[eigenvalues, 16]];
$k = 1.0;


-Graphics-
-Graphics-
-Graphics-
-Graphics-
\:7279\:5f81\:503c (\:5e26\:6709\:6570\:503c\:7cbe\:5ea6):
{9.42461, 12.5663, 15.7079}


mma版本，粗略查找区间，再精确查询，效果并不好

In [126]:
(* 特征值求解函数 *)
shootEV[] := 
  Module[{tol = 10^-8, m = 3, kk, n, dk, x, phi, oldphi, dphi, evFun, sol,
     currentGuess, kHistory, eigenvalueMarkers, initialSearchSteps = 50},
   (* 定义微分方程 *)
   evFun[x_, phi_] := {phi[[2]], -$k^2*phi[[1]]};
   (* 预分配特征值数组 *)
   kk = ConstantArray[0.0, m];
   eigenvalueMarkers = {};(* 用于存储特征值位置 *)
   (* 循环查找前'm'个特征值 *)
   n = 1;
   currentGuess = 1.0;(* 从1.0开始搜索 *)
   While[n <= m,
    (* --- 初始搜索阶段（更系统化） --- *)
    kHistory = {};
    dk = 0.1;(* 初始搜索的较小且恒定的dk。 *)
    oldphi = 10^
      6;(* 在内循环*之前*初始化oldphi *)
    While[True,
     AppendTo[kHistory, currentGuess];
     sol = Quiet@
       Check[NDSolve[{y''[x] == -currentGuess^2*y[x], y[0] == 0, 
           y'[0] == 10^-3}, y, {x, 0, 1}, 
          Method -> "StiffnessSwitching"], $Failed, {NDSolve::ndsz, 
          NDSolve::ndcf}];
     If[sol === $Failed, phi = {10^6, 10^6};, phi = y /. sol[[1]];];
     If[Head[phi] === InterpolatingFunction, dphi = phi[1];, 
      dphi = 10^6;];
     If[dphi*oldphi < 
        0,(* 检测到符号变化！ *)
      Break[];(* 退出初始搜索循环 *)
      ];
     oldphi = dphi;(* 在初始搜索循环*内部*更新oldphi *)
     
     currentGuess += 
      dk;(* 初始搜索的k增量。 *)
     If[currentGuess > 20,(* 防止无限循环 *)
      Print["警告：未找到符号变化。结果可能不完整。"];
      Return[{$Failed, 
        "警告：未找到符号变化。"}];(*修改为返回一个列表*)
      ];
     ];(* While True结束 *)
    (* --- 二分法阶段（精炼） --- *)
    dk = dk/2.0;(* 减少二分法的步长 *)
    While[Abs[dphi] > tol,
     currentGuess = currentGuess + dk;
     AppendTo[kHistory, currentGuess];
     sol = Quiet@
       Check[NDSolve[{y''[x] == -currentGuess^2*y[x], y[0] == 0, 
           y'[0] == 10^-3}, y, {x, 0, 1}, 
          Method -> "StiffnessSwitching"], $Failed, {NDSolve::ndsz, 
          NDSolve::ndcf}];
     If[sol === $Failed, phi = {10^6, 10^6};, phi = y /. sol[[1]];];
     If[Head[phi] === InterpolatingFunction, dphi = phi[1];, 
      dphi = 10^6;];
     If[dphi*oldphi < 0,
      currentGuess = currentGuess - dk;
      AppendTo[kHistory, currentGuess];
      dk = dk/2.0;
      ];
     oldphi = dphi;
     ];(* While Abs[dphi] > tol结束 *)
    kk[[n]] = currentGuess;(* 存储特征值 *)
    AppendTo[eigenvalueMarkers, {currentGuess, 0}];

    (* 绘制k历史 *)
    Print[ListLinePlot[kHistory, 
      AxesLabel -> {"迭代", "k"}, 
      PlotLabel -> 
       StringJoin["特征值 ", ToString[n], " 的搜索历史"], 
      PlotRange -> All, 
      Epilog -> {Red, PointSize[Large], 
        Point[{Length[kHistory], currentGuess}]}]];

    (* --- 准备下一个特征值搜索 --- *)
    currentGuess = kk[[n]] + 0.1;(* 正确的增量 *)
    n++;(* 在存储特征值后递增n *)
    ];(* While[n <= m]结束 *)
   (* --- 绘制组合特征函数 --- *)
   plot = Plot[
     Evaluate[
      Table[If[
        Head[y /. 
           NDSolve[{y''[x] == -kk[[i]]^2*y[x], y[0] == 0, 
              y'[0] == 10^-3}, y, {x, 0, 1}, 
             Method -> "StiffnessSwitching"][[1]]] === 
         InterpolatingFunction, (y /. 
           NDSolve[{y''[x] == -kk[[i]]^2*y[x], y[0] == 0, 
              y'[0] == 10^-3}, y, {x, 0, 1}, 
             Method -> "StiffnessSwitching"][[1]])[x], 
        Null], {i, 1, m}]], {x, 0, 1}, PlotRange -> All, 
     AxesLabel -> {"x", "phi(x)"}, PlotLabel -> "特征函数", 
     GridLines -> Automatic, 
     GridLinesStyle -> Directive[GrayLevel[0.8]]];
    
   Return[{kk, plot}]; (*返回一个列表*)
   ];

(* 调用函数并显示结果 *)
{eigenvalues,plot} = shootEV[];

If[eigenvalues=!=$Failed,
Print["特征值（数值精度）："];
Print[N[eigenvalues, 16]];
Print[plot];,
Print[plot];
]


-Graphics-
-Graphics-
-Graphics-
\:7279\:5f81\:503c\:ff08\:6570\:503c\:7cbe\:5ea6\:ff09\:ff1a
{9.42461, 12.566, 21.991}
-Graphics-
